# Pré-traitement du dataset recipes

Dataset issu de kaggle : https://www.kaggle.com/datasets/thedevastator/better-recipes-for-a-better-life?resource=download

In [29]:
import numpy as np
import pandas as pd
import json

def show_df(df):
    print(df.shape)
    display(pd.concat([df.head(2), df.tail(1)]))

In [30]:
df = pd.read_csv("/home/camille/code/Camille9999/EPSI/workshop/workshop-m1/data/kaggle/recipes.csv")
show_df(df)

(1090, 15)


,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
1089,1089,Chef John's Chocolate Energy Bars,15 mins,NaN,3 hrs,12,12 bars,"2 cups pitted Medjool dates, roughly chopped, ...","Place cashews, almonds, coconut, dates, cocoa,...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,/Appetizers and Snacks/Snacks/Granola Bar Reci...,"Total Fat 22g 28%, Saturated Fat 7g 36%, Sodiu...","Prep Time: 15 mins, Additional Time: 2 hrs 45 ...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...


In [31]:
import uuid
import re
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download();

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


## Parser ingrédients

In [32]:
'¼', '½', '¾', '⅓', '⅔', '⅛', '⅜', '⅝', '⅞'

('¼', '½', '¾', '⅓', '⅔', '⅛', '⅜', '⅝', '⅞')

In [33]:
import re
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

def compact_numbers_in_string(s):
    def compact_match(match):
        numbers = match.group().split()
        total = sum(float(num) for num in numbers)
        return str(total)
    compacted_string = re.sub(r'(\d+\.\d+|\d+)(\s+(\d+\.\d+|\d+))+', compact_match, s)
    return compacted_string

def fraction_to_decimal(text):
    def replace_fraction(match):
        numerator, denominator = map(int, match.group().split('/'))
        decimal_value = round(numerator / denominator, 3)
        return f' {decimal_value} '
    fraction_pattern = re.compile(r' (?P<numerator>\d+)/(?P<denominator>\d+) ')
    return fraction_pattern.sub(replace_fraction, text)

def convert_fraction_to_decimal(fraction_str: str):
    unicode_fractions = {
        '¼': '0.25',
        '½': '0.5',
        '¾': '0.75',
        '⅓': '0.33',
        '⅔': '0.67',
        '⅛': '0.125',
        '⅜': '0.375',
        '⅝': '0.625',
        '⅞': '0.875'
    }
    for key, val in unicode_fractions.items():
        fraction_str = fraction_str.replace(key, val)
    fraction_str = compact_numbers_in_string(fraction_str)
    fraction_str = fraction_to_decimal(fraction_str)
    return fraction_str

def remove_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text)

def split_ingredients(ingredient_list):
    return re.split(r',\s*(?=\d)', ingredient_list)

def convert_to_metric(amount, unit):
    conversion_factors = {
        'teaspoon': 4.92892,
        'teaspoons': 4.92892,
        'tablespoon': 14.7868,
        'tablespoons': 14.7868,
        'cup': 240,
        'cups': 240,
        'ounce': 28.3495,
        'ounces': 28.3495,
        'pint': 473.176,
        'pints': 473.176,
        'quart': 946.353,
        'quarts': 946.353,
        'gallon': 3785.41,
        'gallons': 3785.41,
        'pound': 453.592,
        'pounds': 453.592
    }

    if unit in conversion_factors:
        metric_amount = float(amount) * conversion_factors[unit]
        if unit in ['pound', 'pounds', 'ounce', 'ounces']:
            return round(metric_amount), 'g'
        else:
            return round(metric_amount), 'ml'
    return amount, None

def extract_ingredients(ingredient_list):
    ingredient_list = convert_fraction_to_decimal(ingredient_list)
    ingredient_list = remove_parentheses(ingredient_list)
    ingredient_parts = split_ingredients(ingredient_list)

    common_units = {'teaspoon', 'teaspoons', 'tablespoon', 'tablespoons', 'cup', 'cups',
                    'ounce', 'ounces', 'pint', 'pints', 'quart', 'quarts',
                    'gallon', 'gallons', 'pound', 'pounds'}

    ingredients = {}
    for part in ingredient_parts:
        tokens = word_tokenize(part)
        tagged = pos_tag(tokens)
        amount = convert_fraction_to_decimal(tagged[0][0])
        unit = None
        ingredient = []
        for word, tag in tagged:
            if tag in ['NN', 'NNS', 'JJ']:  # Noun or adjective
                if word.lower() in common_units:
                    unit = word
                else:
                    ingredient.append(word)
        if amount is not None and ingredient:
            ingredient_name = ' '.join(ingredient).strip()
            try:
                metric_amount, metric_unit = convert_to_metric(amount, unit)
            except Exception as e:
                print(f"Erreur de conversion: {e}\n - ingredient_list : {ingredient_list},\n - ingredient : {part}")
                metric_amount, metric_unit = amount, unit
            try:
                metric_amount = float(metric_amount)
            except:
                metric_amount = 1
            ingredients[ingredient_name] = {"amount" : metric_amount, "unit" : metric_unit}

    return ingredients

# Example ingredient lists
ingredient_list1 = "3 tablespoons butter, 2 pounds Granny Smith apples (or other firm, crisp apples), peeled, quartered, cored and sliced 1/4-inch thick, 1 pound Macintosh apples (or other soft-textured apples that fall apart when cooked), peeled, quartered, cored, and sliced 1/4-inch thick, ½ cup sugar, ½ cup dried cranberries, 1 sheet frozen puff pastry, thawed but still cold (follow package directions), 1 egg white, lightly beaten, 1 tablespoon sugar, 1 cup Optional: Ice cream or lightly sweetened whipped cream"
ingredient_list2 = "2 tablespoons butter, 3 skinless, boneless chicken breast halves, 1 ½ teaspoons salt, 1 ½ teaspoons ground black pepper, 2 tablespoons garlic powder, divided, 1 lemon, juiced"

print(extract_ingredients(ingredient_list1))
print(extract_ingredients(ingredient_list2))

{'butter': {'amount': 44.0, 'unit': 'ml'}, 'apples 1/4-inch thick': {'amount': 454.0, 'unit': 'g'}, 'sugar': {'amount': 15.0, 'unit': 'ml'}, 'cranberries': {'amount': 120.0, 'unit': 'ml'}, 'sheet frozen puff pastry': {'amount': 1.0, 'unit': None}, 'egg white': {'amount': 1.0, 'unit': None}, 'cream cream': {'amount': 240.0, 'unit': 'ml'}}
{'butter': {'amount': 30.0, 'unit': 'ml'}, 'skinless boneless chicken breast halves': {'amount': 3.0, 'unit': None}, 'salt': {'amount': 7.0, 'unit': 'ml'}, 'ground black pepper': {'amount': 7.0, 'unit': 'ml'}, 'garlic powder': {'amount': 30.0, 'unit': 'ml'}, 'lemon': {'amount': 1.0, 'unit': None}}


## Parser nutrition

In [34]:
def extract_nutrition(nutritional_values):
    entries = nutritional_values.split(', ')
    nutrition_dict = {}
    for entry in entries:
        parts = entry.split(' ')
        name = ' '.join(parts[:-2]) if '%' in parts[-1] else ' '.join(parts[:-1])
        amount = parts[-2] if '%' in parts[-1] else parts[-1]
        percent = parts[-1] if '%' in parts[-1] else None
        nutrition_dict[name] = {'amount': amount, 'percent': percent}
    return nutrition_dict


nutritional_values = 'Total Fat 0g 0%, Sodium 24mg 1%, Total Carbohydrate 41g 15%, Dietary Fiber 2g 5%, Total Sugars 22g, Protein 1g, Vitamin C 30mg 148%, Calcium 23mg 2%, Iron 1mg 3%, Potassium 228mg 5%'
print(extract_nutrition(nutritional_values))

nutritional_values2 = 'Total Fat 19g 24%, Saturated Fat 9g 46%, Cholesterol 31mg 10%, Sodium 124mg 5%, Total Carbohydrate 52g 19%, Dietary Fiber 3g 10%, Protein 2g, Vitamin C 5mg 27%, Calcium 23mg 2%, Iron 1mg 6%, Potassium 156mg 3%'
print(extract_nutrition(nutritional_values2))


{'Total Fat': {'amount': '0g', 'percent': '0%'}, 'Sodium': {'amount': '24mg', 'percent': '1%'}, 'Total Carbohydrate': {'amount': '41g', 'percent': '15%'}, 'Dietary Fiber': {'amount': '2g', 'percent': '5%'}, 'Total Sugars': {'amount': '22g', 'percent': None}, 'Protein': {'amount': '1g', 'percent': None}, 'Vitamin C': {'amount': '30mg', 'percent': '148%'}, 'Calcium': {'amount': '23mg', 'percent': '2%'}, 'Iron': {'amount': '1mg', 'percent': '3%'}, 'Potassium': {'amount': '228mg', 'percent': '5%'}}
{'Total Fat': {'amount': '19g', 'percent': '24%'}, 'Saturated Fat': {'amount': '9g', 'percent': '46%'}, 'Cholesterol': {'amount': '31mg', 'percent': '10%'}, 'Sodium': {'amount': '124mg', 'percent': '5%'}, 'Total Carbohydrate': {'amount': '52g', 'percent': '19%'}, 'Dietary Fiber': {'amount': '3g', 'percent': '10%'}, 'Protein': {'amount': '2g', 'percent': None}, 'Vitamin C': {'amount': '5mg', 'percent': '27%'}, 'Calcium': {'amount': '23mg', 'percent': '2%'}, 'Iron': {'amount': '1mg', 'percent': '6

## Parse time

In [35]:
def extract_time(time_str):
    parts = time_str.split()
    total_minutes = 0
    for i in range(0, len(parts), 2):
        value = int(parts[i])
        unit = parts[i+1]
        if unit.startswith('hr'):
            total_minutes += value * 60
        elif unit.startswith('min'):
            total_minutes += value
    return total_minutes

time1 = "1 hrs 30 mins"
time2 = "25 mins"
extract_time(time1), extract_time(time2)

(90, 25)

## Cleaning du dataset

In [47]:
df_clean = df.copy().drop(columns=["Unnamed: 0", "yield", "cook_time", "prep_time", "timing"]).dropna(subset='total_time').reset_index(drop=True)
df_clean.ingredients = df_clean.ingredients.apply(extract_ingredients)
df_clean.nutrition = df_clean.nutrition.apply(extract_nutrition)
df_clean.total_time = df_clean.total_time.apply(extract_time)
df_clean = df_clean.rename(columns={'total_time' : 'total_time_minutes'})
df_clean["uuid"] = df_clean.apply(lambda _: str(uuid.uuid4()), axis=1)
show_df(df_clean)

(1045, 11)


,recipe_name,total_time_minutes,servings,ingredients,directions,rating,url,cuisine_path,nutrition,img_src,uuid
0,Apple Pie by Grandma Ople,90,8,"{'small apples': {'amount': 8.0, 'unit': None}...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"{'Total Fat': {'amount': '19g', 'percent': '24...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,f01e0c4c-42db-4fcf-b599-b8bf9e3f6c5c
1,Sarah's Homemade Applesauce,25,4,"{'apples': {'amount': 4.0, 'unit': None}, 'wat...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"{'Total Fat': {'amount': '0g', 'percent': '0%'...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,d9efa0fb-d2e6-47d4-90d7-7cc14056afb3
1044,Chef John's Chocolate Energy Bars,180,12,"{'dates': {'amount': 480.0, 'unit': 'ml'}, 'ra...","Place cashews, almonds, coconut, dates, cocoa,...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,/Appetizers and Snacks/Snacks/Granola Bar Reci...,"{'Total Fat': {'amount': '22g', 'percent': '28...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...,bd72f216-3238-4f23-9088-2e8fed12dd96


## Identification des régimes compatibles et restrictions

In [52]:
meats = [
    "chicken", "beef", "pork", "lamb", "turkey", "duck", "goat",
    "veal", "bacon", "ham", "sausage", "salami", "pepperoni",
    "prosciutto", "venison", "rabbit", "quail", "pheasant",
    "buffalo", "bison", "ostrich", "kangaroo"
]

fish_and_seafood = [
    "salmon", "tuna", "cod", "haddock", "trout", "mackerel",
    "sardines", "anchovies", "halibut", "snapper", "bass",
    "catfish", "tilapia", "flounder", "sole", "grouper",
    "swordfish", "shark", "eel", "herring", "pollock",
    "crab", "lobster", "shrimp", "prawns", "scallops",
    "clams", "mussels", "oysters", "squid", "octopus"
]


diet_keywords = {
    "vegan": meats + fish_and_seafood + ["meat", "fish", "dairy", "egg", "butter", "cheese", "milk", "honey"],
    "vegetarian": meats + fish_and_seafood + ["meat", "fish"],
    "pescitarian": meats + ["meat"],
    "omnivore": []
}

allergen_keywords = {
    "eggs": ["egg"],
    "dairy": ["butter", "milk", "cheese"],
    "nuts": ["nuts", "cashew", "almond", "walnut", "pecan", "pistachio", "hazelnut", "macadamia", "brazil nut", "pine nut", "chestnut"],
    "soy": ["soy"]
}

gluten_keywords = ["flour", "wheat", "barley", "rye"]
fodmap_keywords = ["apple", "honey", "garlic", "onion", "wheat"]


def identify_diet(ingredients):
    for diet, keywords in diet_keywords.items():
        if all(not any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in keywords) for ingredient in ingredients):
            return diet
    return "omnivore"


def identify_tags(ingredients):
    tags = []
    for allergen, keywords in allergen_keywords.items():
        if any(any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in keywords) for ingredient in ingredients):
            tags.append(f"contains_{allergen}")
    if any(any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in gluten_keywords) for ingredient in ingredients):
        tags.append("contains_gluten")
    if any(any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in fodmap_keywords) for ingredient in ingredients):
        tags.append("high_FODMAP")
    return tags

ingredient_names = df_clean.at[0, 'ingredients'].keys()

diet = identify_diet(ingredient_names)
tags = identify_tags(ingredient_names)
print(f"Diet Compatibility: {diet}")
print(f"Tags: {tags}")


Diet Compatibility: vegetarian
Tags: ['contains_dairy', 'contains_gluten', 'high_FODMAP']


In [53]:
df_clean["diet"] = df_clean.ingredients.apply(identify_diet)
df_clean["tags"] = df_clean.ingredients.apply(identify_tags)
show_df(df_clean)

(1045, 13)


,recipe_name,total_time_minutes,servings,ingredients,directions,rating,url,cuisine_path,nutrition,img_src,uuid,diet,tags
0,Apple Pie by Grandma Ople,90,8,"{'small apples': {'amount': 8.0, 'unit': None}...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"{'Total Fat': {'amount': '19g', 'percent': '24...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,f01e0c4c-42db-4fcf-b599-b8bf9e3f6c5c,vegetarian,"[contains_dairy, contains_gluten, high_FODMAP]"
1,Sarah's Homemade Applesauce,25,4,"{'apples': {'amount': 4.0, 'unit': None}, 'wat...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"{'Total Fat': {'amount': '0g', 'percent': '0%'...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,d9efa0fb-d2e6-47d4-90d7-7cc14056afb3,vegan,[high_FODMAP]
1044,Chef John's Chocolate Energy Bars,180,12,"{'dates': {'amount': 480.0, 'unit': 'ml'}, 'ra...","Place cashews, almonds, coconut, dates, cocoa,...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,/Appetizers and Snacks/Snacks/Granola Bar Reci...,"{'Total Fat': {'amount': '22g', 'percent': '28...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...,bd72f216-3238-4f23-9088-2e8fed12dd96,vegan,[contains_nuts]


In [57]:
df_clean.cuisine_path = df_clean.cuisine_path.apply(lambda x: x.split('/')[1])
df_clean = df_clean.rename(columns={'cuisine_path' : 'recipe_type'})

## Export du dataframe au format json

In [59]:
result_dict = df_clean.set_index('uuid').to_dict(orient='index')

with open('../data/recipes.json', 'w') as json_file:
    json.dump(result_dict, json_file)

### Format recipes.json

**Clé :** *uuid* de la recette

**Valeurs :**

| Valeur | Nom technique | Format |
|--------|---------------|--------|
| Nom | recipe_name | str |
| Temps de préparation | total_time_minutes | int |
| Parts | servings | int |
| Ingrédients | ingredients | dict |
| Instructions | directions | str |
| Note | rating | float |
| Lien | url | str |
| Nutrition | nutrition | dict |
| Image | img_src | str |
| Régime compatible | diet | str |
| Restrictions alimentaires | tags | list[str] |
| Type de plat | recipe_type | str |

**Format ingrédients :** {
  nom : {
    'amount' : float, 
    'unit' : str | None
  },
  ...
}

**Format nutrition :** { 
  nom : {
    'amount' : str (quantité + unité), 
    'percent' : str (quantité%, % de la dose journalière recommandée)
  },
  ...
}

